In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../keys.env")
openai_api_key = os.getenv("GRAVY_LAB_OPENAI")

In [2]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings = OpenAIEmbeddings(api_key=openai_api_key, model="text-embedding-3-large")

"""
"hello world"라는 문장의 임베딩 길이를 제공하는 이유는 사용 중인 임베딩 모델이 반환하는 벡터의 차원을 알아내기 위한 방법임.

- faiss.IndexFlatIP : 내적 계산을 사용해 벡터간 유사성을 측정하는 인덱스
- faiss.IndexFlatL2 : L2 거리를 사용해 벡터간 유사성을 측정하는 인덱스
- faiss.IndexHNSWFlat : HNSW 알고리즘을 사용해 벡터간 유사성을 측정하는 인덱스
"""
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [3]:
import mysql.connector
from mysql.connector import Error

def fetch_data_from_mysql():
    try:
        # MySQL 데이터베이스 연결
        connection = mysql.connector.connect(
            host="ls-662e72c42a259736c11c93759076a7409118e05a.cgzykv9hk4gr.ap-northeast-2.rds.amazonaws.com",
            port=3306,
            user="gravylab",
            password="glab0110!!",
            database="recruitdb"
        )

        if connection.is_connected():
            print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
            
            query = """
                SELECT * 
                FROM recruit
                WHERE platform_type IN ('WANTED');
            """
            cursor = connection.cursor(dictionary=True)
            cursor.execute(query)
            
            results = cursor.fetchall()

            cursor.close()
            connection.close()
            print("MySQL 연결이 닫혔습니다.")

            return results

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")

In [4]:
dataset = fetch_data_from_mysql()
print(len(dataset))

MySQL 데이터베이스에 성공적으로 연결되었습니다.
MySQL 연결이 닫혔습니다.
11115


In [5]:
from uuid import uuid4
from langchain_core.documents import Document

def dataset_to_documents(dataset, page_content_fields, metadata_fields):
    documents = []
    for data in dataset:
        page_content = ""
        for field in page_content_fields:
            page_content += f"{field}\n{data[field]}\n\n"

        metadata = {}
        for field in metadata_fields:
            metadata[field] = data[field]
            
        documents.append(Document(page_content=page_content, metadata=metadata, id=uuid4()))

    return documents

In [6]:
page_content_fields = ['title', 'description', 'tasks', 'requirements', 'points', 'work_description'] 
metadata_fields = ['career_type', 'career_min', 'career_max', 'work_type', 'education_type', 'workday_content', 'info_url']

documents = dataset_to_documents(dataset, page_content_fields, metadata_fields)
print(len(documents))

11115


In [7]:
from random import randint

idx = randint(0, len(documents))
print(f"Index: {idx}")

sample_doc = documents[idx]
print(f"Page Content:\n{sample_doc.page_content}\n\n")
print(f"Metadata:\n{sample_doc.metadata}")

Index: 3761
Page Content:
title
[Photoism][IPX-MD] MD 사업관리 담당자(책임매니저)

description
서북의 핵심가치는 '누구나 즐길 수 있는 문화를 만드는 것'입니다.
새로운 문화를 만들고, 물리적이고 문화적인 장벽을 허무는 것, 우리가 가장 잘하는 것입니다.
On-Offline을 잇는 허브가 되고자 하는 우리는 동종업계 성장률 1위를 자랑하고 있으며, 국내 뿐만 아니라 해외로 진출하여 현재 12개국 120여개 이상의 가맹점을 운영하고 있습니다.
Global entertainment platform으로써 지금 이 순간도 성장하는 서북의 로켓에 탑승할 크루를 찾습니다 !

서북의 대표 프로덕트는 포토이즘으로 국내 450여 개, 글로벌 150여 개의 오프라인 매장을 운영 중이며 특히, 글로벌은 가파른 성장세를 보이고 있어 2025년부터는 글로벌을 중심으로 사업을 확장할 계획입니다.

tasks
• MD관련 사업성 검토 및 사업 전개, MD파트너십 계약 및 관리
• 파트너사 협의를 통한 세일즈 확대 전략 수립 및 실행
• 신규 비즈니스 모델(BM) 확장 제안, 시장 트렌드 분석 및 기회 발굴

requirements
• MD 사업 실행 및 관리 경험, 팝업스토어 전개 및 관리 경험
  (최소 3년 이상, 5년 이상 우대)
• 실무 경험을 바탕으로 하는 IP협업 및 엔터테인먼트 산업에 대한 높은 이해도
• 신규 비즈니스 모델 개발 및 확장에 대한 실무 경험
• 세일즈 및 파트너십 확장 전략 수립 능력
• 뛰어난 커뮤니케이션 및 협상 능력

points
• 엔터테인먼트 또는 MD(머천다이징) 관련 경력
• 팬덤 경제 및 IP비즈니스에 대한 이해
• 다양한 IP와의 파트너십 성공 경험
• 팝업스토어 운영, 계약 등 관련 업무경험 우대

work_description
• MD관련 사업성 검토 및 사업 전개, MD파트너십 계약 및 관리
• 파트너사 협의를 통한 세일즈 확대 전략 수립 및 실행
• 신규 비즈니스 모델(BM) 확장

In [9]:
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

print(vector_store)

In [12]:
results = vector_store.similarity_search_with_score("프론트엔드 개발자", k=5)

for res, score in results:
    print(score)
    print(res.metadata)
    print(res.page_content)
    print("-"*100)

0.9029131
{'career_type': 'EXPERIENCED', 'career_min': 3, 'career_max': 3, 'work_type': 'FULL_TIME', 'education_type': 'NO_PREFERENCE', 'workday_content': '주 5일제', 'info_url': 'https://www.wanted.co.kr/wd/255411'}
title
프론트엔드 개발자

description
• pre-A EduTech 스타트업 트루밸류에서 프론트엔드 개발자를 모집합니다.
• 트루밸류는 이번 투자유치를 통해 본격적인 성장의 발판을 준비 중에 있으며 그 중 개발팀 빌딩은 새로운 성장 동력 확보에 있어 가장 핵심적인 퍼즐 조각으로 인지하고 있습니다.
• 서로의 꿈을 응원하고 꿈을 이뤄가는 과정을 공유하는 Z세대를 위한 SNS 드림어필과 진로교육플랫폼 에버멘토의 iOS, Andrord, Web 개발 및 유지보수를 담당해줄 프론트엔드 개발자를 모집합니다.

tasks
• 기획자와 협업하여 플랫폼 설계, 구축, 운영
• 확장성, 테스트 자동화가 가능하도록 아키텍쳐 설계 및 개선
• React, React native 기반 웹, 앱 개발
• RESTful 기반 API 연동

requirements
• 프론트엔드 개발 경력 3년 이상
• HTML, CSS, JS 언어에 대한 깊은 이해
• Next.js, react 기반의 웹서버 개발 경험 보유자
• AWS 환경 구축/운영 경험이 있으신 분(ec2, nginx, ssl, https)
• react native, expo 기반의 앱 개발 경험 보유자( 심사 및 배포 경험 필수)
• 협업툴(Git, GitHub action) 및 빌드관리도구사용경험 보유자
• Backend, DB 개발에 대한 지식보유자

points
• 특정 언어, 프레임워크에 구애 받지 않는 우수한 학습 능력이 있으신 분
• 협업 및 문서 읽기/쓰기에 지장이 없는 영어 구사 능력이 있으신 분
• 우수한 문서 커뮤니케이션